# Testing BERT Model

Since we will use Bert to embed document contexts, we need to test bert model. \
As namuwiki is based on Korean, we will use KOBert(distilled ver, <https://github.com/monologg/DistilKoBERT>) \
This Code is tested on Colab, So it might be not available on other environments.



In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers

In [2]:
!git clone https://github.com/monologg/DistilKoBERT

%cd DistilKoBERT

fatal: destination path 'DistilKoBERT' already exists and is not an empty directory.
/content/DistilKoBERT


In [3]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-08-07 19:19:59--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-08-07 19:19:59--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce37f4ae2b18e17b8d9f08d98ca.dl.dropboxusercontent.com/cd/0/get/BTyVBc-3BiYHyQZVkUTjabD55k23x-xkYyFzK40nviaXf53-adv68ULMbNc5sClBEYxPAQzgR3BTCS3rOcjT3aXeJThNfOu01v72Lg05WPI5dWoG2aA_UlWDEseC1XvJ0GD5YoXI-jlEG4PXRWR9vu08/file?dl=1# [following]
--2021-08-07 19:19:59--  https://uce37f4ae2b18e17b8d9f08d98ca.dl.dropboxusercontent.com/cd/0/get/BTyVBc-3BiYHyQZVkUTjabD55k23x-xkYyFzK40nviaXf53-adv68ULMbNc5sClBEYxPAQzgR

# Load Libraries

Using transformers, we can load model easily.

In [4]:
import gluonnlp as nlp
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tokenization_kobert import KoBertTokenizer
from transformers import DistilBertModel

Load Pretrained KoBert Model

In [5]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
bert = DistilBertModel.from_pretrained('monologg/distilkobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClass

In [6]:
# Use tokenizer, tokenize given string
# By use tokenizer.encode we can encoding string to token-idx and can easily padding

test = '한국어 모델을 공유합니다.'

token = tokenizer.encode(test)

token

[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [7]:
# Read data

dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

dataset_test[0]

['굳 ㅋ', '1']

# Make Dataset

To train model, we have to process original data with tokenizer. \
For this propose, use Dataset class in pyotrch.

In [8]:
class Rating(Dataset):
  def __init__(self, data, tokenizer, max_len):
    self.comments = [tokenizer(d[0], padding='max_length', max_length=max_len, return_tensors= 'pt', truncation=True) for d in data]
    self.labels = [d[1] for d in data]

  def __getitem__(self, idx):
    return self.comments[idx], torch.tensor([int(self.labels[idx])])

  def __len__(self):
    return len(self.labels)

In [9]:
# Trian settings
epoch = 5
max_len = 64
learning_rate = 0.001
batch_size = 128
log_interval = 500
max_grad_norm = 0.5

device = 'cuda'

In [10]:
dataset_train = Rating(dataset_train, tokenizer, max_len)
dataset_test = Rating(dataset_test, tokenizer, max_len)

trainLoader = DataLoader(dataset_train, batch_size=batch_size, num_workers=5)
testLoader = DataLoader(dataset_test, batch_size=batch_size, num_workers=5)

dataset_test[0]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


({'input_ids': tensor([[   2,  517, 5515,  517,  492,    3,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}, tensor([1]))

In [11]:
# To test tokenizer works correct, reverse data

tokenizer.convert_ids_to_tokens(dataset_test[0][0]['input_ids'][0])

['[CLS]',
 '▁',
 '굳',
 '▁',
 'ᄏ',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [12]:
# Test model

output = bert(input_ids=dataset_test[0][0]['input_ids'], attention_mask=dataset_test[0][0]['attention_mask'])

# Output of DistillBert is sequence_output, (hidden_states), (attentions)
# To get embeddign of [CLS] token, sequence_output[0][:, 0]
output[0][:, 0].size()

torch.Size([1, 768])

# Define Mdoel

Bert model itself is not predication model. It is more like embedding model. \
To predict review rating, we need to construct classification model based on Bert.

In [13]:
class Classifier(nn.Module):
  def __init__(self, bert, hidden_layers=[512], num_classes=2):
    """
    Parameters
    --------------

    bert : Bert model
      Bert Model to imbed input text
    hidden_layers : list(int)
      Layers of classifier, each int is size of each layers
    num_classes : int
      output classes
    """
    super(Classifier, self).__init__()
    self.bert = bert

    layers = []
    # Output size of bert model
    input_size = 768

    layers += [nn.BatchNorm1d(input_size),
               nn.ReLU(inplace=True)]

    for layer_size in hidden_layers:
      layers += [nn.Linear(input_size, layer_size),
                          nn.BatchNorm1d(layer_size),
                          nn.ReLU(inplace=True),
                          nn.Dropout(p=0.5)]

      input_size = layer_size
    
    layers.append(nn.Linear(input_size, num_classes))

    self.hidden = nn.Sequential(*layers)


  def forward(self, input_ids, attention_mask):
    x = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0][:, 0]

    x = self.hidden(x)

    return x
    

In [14]:
model = Classifier(bert).to(device)
loss_fn = nn.CrossEntropyLoss()

# Train Model


In [15]:
opt = optim.AdamW(model.parameters(), lr=learning_rate)

In [16]:
def Cal_accuracy(pred, y):
  pred_label = torch.argmax(pred, dim=1)
  acc = (pred_label == y).sum().data.cpu() / (len(y) + 1)
  return acc

In [18]:
# To plot loss save losses
train_loss = []
test_loss = []

for e in range(epoch):
  model.train()
 
  i = 0
  for inputs, labels in trainLoader:
    opt.zero_grad()

    input_ids = inputs['input_ids'].squeeze().to(device)
    attention_mask = inputs['attention_mask'].squeeze().to(device)

    labels = labels.squeeze().to(device)

    pred = model(input_ids, attention_mask)
    loss = loss_fn(pred, labels)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    opt.step()

    train_loss.append(loss.mean().data.cpu())
    acc = Cal_accuracy(pred, labels)

    i += 1
    if (i % log_interval) == 0:
      print(f"epoch {e} train_loss {loss.data.cpu()} acc {acc}")
  
  model.eval()
  i = 0
  losses = 0
  acc = 0
  for inputs, labels in testLoader:
    input_ids = inputs['input_ids'].squeeze().to(device)
    attention_mask = inputs['attention_mask'].squeeze().to(device)

    labels = labels.squeeze().to(device)

    pred = model(input_ids, attention_mask)
    loss = loss_fn(pred, labels)
    losses += loss.sum().data.cpu()
    acc += Cal_accuracy(pred, labels)

    test_loss.append(loss.mean().data.cpu())
    i += 1
  
  losses = losses / (batch_size * i)
  acc = acc / i

  print(f"epoch {e} test_loss {losses} acc {acc}")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 0 train_loss 0.3301074206829071 acc 0.8449612259864807
epoch 0 train_loss 0.24193592369556427 acc 0.8759689927101135
epoch 0 test_loss 0.0027106255292892456 acc 0.8405233025550842
epoch 1 train_loss 0.29001665115356445 acc 0.8682170510292053
epoch 1 train_loss 0.18155460059642792 acc 0.9224806427955627
epoch 1 test_loss 0.002999036107212305 acc 0.834829568862915
epoch 2 train_loss 0.24404986202716827 acc 0.8914728760719299
epoch 2 train_loss 0.15430374443531036 acc 0.9457364082336426
epoch 2 test_loss 0.003221197286620736 acc 0.8367566466331482
epoch 3 train_loss 0.1998506486415863 acc 0.9379844665527344
epoch 3 train_loss 0.1112002432346344 acc 0.9534883499145508
epoch 3 test_loss 0.004358542151749134 acc 0.8123865127563477
epoch 4 train_loss 0.14493706822395325 acc 0.930232584476471
epoch 4 train_loss 0.06852833926677704 acc 0.9689922332763672
epoch 4 test_loss 0.003728030016645789 acc 0.8272156119346619
